# Phi 3 tool calling example with Duck Duck Go

7/8/24

Credit to [@joshuasundance-swca](https://github.com/joshuasundance-swca)

This notebook is a first attempt at getting Phi3 to use tools, running with `mistralrs`.

```bash
mistralrs serve -p 8099 --isq Q4K -m microsoft/Phi-3-mini-128k-instruct
```

```bash
python -m pip install openai duckduckgo-search langchain langchain_community langchain_openai
```

It's pretty easy because `mistralrs` is `openai`-compatible, and `langchain_openai.ChatOpenAI` doesn't actually care where you send the requests.


In [1]:
from langchain.agents import AgentExecutor
from langchain.agents import create_json_chat_agent
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    openai_api_base="http://localhost:8099/v1",
    openai_api_key="EMPTY",
    streaming=True,
    temperature=0.50,
    verbose=True,
)


ddg_wrapper = DuckDuckGoSearchAPIWrapper(max_results=5)
ddg_search = DuckDuckGoSearchResults(api_wrapper=ddg_wrapper)
tools = [ddg_search]


system_message_content = """You are Phi, a powerful AI agent built by Microsoft. In order to effectively serve the user, you have access to the following tools:

# TOOLS

{tools}

# RESPONSE INSTRUCTIONS

## RESPONSE OPTION 1 (TOOL USE)

Use this format to call tools.

Markdown code snippet formatted in the following schema:

```json
{{
    "action": string, \ The action to take. Must be one of {tool_names}
    "action_input": string \ The input to the action
}}
```

## RESPONSE OPTION 2 (FINAL ANSWER)

Use this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:

```json
{{
    "action": "Final Answer",
    "action_input": string \ You should put what you want to return to use here
}}
```"""

human_message_content = """# USER INPUT

Here is the user's input (remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else):

----------------

{input}"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_message_content),
        ("human", human_message_content),
        (
            "human",
            "Use `duckduckgo_results_json` to gather information before answering the question.",
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


agent = create_json_chat_agent(llm, tools, prompt_template)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

ch = {"input": RunnablePassthrough()} | agent_executor | (lambda x: x["output"])

In [2]:
%%time

question = "What is `mistral.rs`?"
ch.invoke(question)



> Entering new AgentExecutor chain...
```json
{
    "action": "duckduckgo_results_json",
    "action_input": "mistral.rs"
}
```[snippet: Architecture-wise, the Idefics 2 model combines a Mistral model, a SigLIP vision encoder, and a Perceiver connector model with a total of just 8B parameters. To get started fast, we provide examples for using our Python, Rust, and HTTP APIs with Idefics 2 here. You can also use our ISQ feature to quantize the Idefics 2 model (there is no llama ..., title: Run the Idefics 2 vision model with mistral.rs: Blazingly fast ... - Reddit, link: https://www.reddit.com/r/rust/comments/1dmddr7/run_the_idefics_2_vision_model_with_mistralrs/], [snippet: Installing Mistral.rs is fairly straightforward, and varies slightly depending on your specific use case. Before getting started, let's get the dependencies out of the way. According to the Mistral.rs README, the only packages we need are libssl-dev and pkg-config. However, we found a few extra packages were nece

"Mistral.rs is a lightweight, fast, and efficient LLM inference platform that supports a wide range of quantization levels, from 2-bit to 8-bit. This allows developers to choose the level of optimization that best suits their needs, balancing inference speed and model accuracy. It also supports device offloading, allowing certain layers of the model to be offloaded to the device for faster inference. Mistral.rs is designed to be highly performant, with equivalent performance to llama.cpp. It provides Python API support with documentation and examples, and it'ors compatible with the OpenAI API and can be used as a standalone HTTP server."

In [3]:
%%time

question = "What is Starcoder-2?"
ch.invoke(question)



> Entering new AgentExecutor chain...
```json
{
    "action": "duckduckgo_results_json",
    "action_input": "Starcoder-2"
}
```[snippet: StarCoder2-7B was trained on 17 programming languages from The Stack v2 on 3.5+ trillion tokens. StarCoder2-15B was trained on 600+ programming languages from The Stack v2 on 4+ trillion tokens. StarCoder2-15B is the best in its size class and matches 33B+ models on many evaluations. StarCoder2-3B matches the performance of StarCoder1-15B:, title: StarCoder2 and The Stack v2 - Hugging Face, link: https://huggingface.co/blog/starcoder2], [snippet: StarCoder2, built by BigCode in collaboration with NVIDIA, is the most advanced code LLM for developers. You can build applications quickly using the model's capabilities, including code completion, auto-fill, advanced code summarization, and relevant code snippet retrievals using natural language. The StarCoder2 family includes 3B, 7B, and ..., title: Unlock Your LLM Coding Potential with StarCoder2, link

'StarCoder2 is a large language model developed by BigCode in collaboration with NVIDIA. It is designed to assist developers by providing capabilities such as code completion, auto-fill, advanced code summarization, and relevant code snippet retrievals using natural language. The StarCoder2 family includes models with 3B, 7B, and higher capacities, with the 7B version being trained on 17 programming languages and the 15B version on over 600 languages, demonstrating high proficiency in code generation and understanding. StarCoder2 is part of the StarCoder project, which aims to advance the development of Large Language Models for Code (Code LLMs).'

In [4]:
%%time

question = "Today is July 8, 2024. Who won the most recent Super Bowl?"
ch.invoke(question)



> Entering new AgentExecutor chain...
```json
{
    "action": "duckduckgo_results_json",
    "action_input": "Who won the most recent Super Bowl as of July 8, 2024?"
}
```[snippet: 49ers vs. Chiefs final score, results, highlights from Super Bowl 58 (All times Eastern) Final: Chiefs 25, 49ers 22 10:47 p.m. TOUCHDOWN — Mecole Hardman scores the touchdown the Chiefs are back ..., title: Who won Super Bowl 2024? - Sporting News, link: https://www.sportingnews.com/us/nfl/news/super-bowl-2024-live-score-49ers-chiefs-results-highlights/0c440aa7145b809ed174d8ff], [snippet: The NFL's overtime rules for the Super Bowl and postseason have evolved over the years, with the most recent tweak coming in 2022. The Super Bowl overtime format is quite similar to what fans are ..., title: Super Bowl 2024 recap: Chiefs top 49ers in OT to repeat as champions ..., link: https://sports.yahoo.com/super-bowl-58-live-updates-130616226.html?fr=sycsrp_catchall], [snippet: Patrick Mahomes rallies the Chiefs to s

"The most recent Super Bowl, as of July 8, 2писанка, was won by the Kansas City Chiefs. Patrick Mahomes, the quarterback for the Chiefs, was named the Most Valuable Player (MVP). The game ended in overtime with a score of Chiefs 25, 49ers 22. This victory marked the Chiefs' second consecutive Super Bowl title. The team's coach, Andy Reid, guided the Chiefs to a successful season, with the team making it to the playoffs after a significant win over the San Francisco 49ers in the AFC Championship game. The Chiefs' offensive strategy, led by Mahomes' exceptional skills, proved to be a significant factor in their victory. Mahomes' performance was not just limited to his passing abilities. He showcased his leadership and decision-making skills on the field, contributing to the team's overall success."

In [5]:
%%time

question = "Who wrote the song, `If You've Got the Money, I've Got the Time`?"
ch.invoke(question)



> Entering new AgentExecutor chain...
```json
{
    "action": "duckduckgo_results_json",
    "action_input": "Who wrote the song If You've Got the Money, I've Got the Time?"
}
```[snippet: Frequently Asked Questions about If You've Got the Money - I've Got the Time 1. Who wrote If You've Got the Money - I've Got the Time? Lefty Frizzell is the songwriter of If You've Got the Money - I've Got the Time. He not only wrote the song but also recorded and released it in 1950. 2. Why did If You've Got the Money - I ..., title: The Meaning Behind The Song: If You've Got the Money - I've Got the ..., link: https://oldtimemusic.com/the-meaning-behind-the-song-if-youve-got-the-money-ive-got-the-time-by-lefty-frizzell/], [snippet: The Meaning Behind The Song: If You've Got the Money I've Got the Time by Willie Nelson. Willie Nelson's song "If You've Got the Money I've Got the Time" was released in 1950 and was Nelson's first top ten hit on the country music charts. The song became an instant cla

"Lefty Frizzell is the songwriter of the song 'If You've Got the Money, I've Got the Time.' He not only wrote the song but also recorded and released it in 1orsm, the user's input is asking for a response to a previous comment without mentioning tool names. Since no specific previous comment content is provided, I'll create a hypothetical response based on a common scenario where a user might ask for clarification or further details. Here's how the response could look:\n\n```json\n{\n    "

In [6]:
%%time

questions = [
    f"What is the current weather in {place}"
    for place in (
        "Phoenix, AZ",
        "Portland, OR",
        "Raleigh, NC",
        "Orlando, FL",
        "Washington, DC",
    )
]

ch.batch(questions)



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...
```json
{
    "action": "duckduckgo_results_json",
    "action_input": "current weather in Phoenix, AZ"
}
``````json
{
    "action": "duckduckgo_results_json",
    "action_input": "current weather Orlando FL"
}
``````json
{
    "action": "duckduckgo_results_json",
    "action_input": "current weather in Raleigh NC"
}
```[snippet: Current conditions at Raleigh / Durham, Raleigh-Durham International Airport (KRDU) Lat: 35.89°NLon: ... Raleigh NC . Tonight. Low: 72 °F. Chance Showers. Monday. High: 92 °F. Mostly Cloudy then Chance ... National Weather Service; Raleigh, NC; 1005 Capability Drive, Suite 300; Centennial Campus ; Raleigh, NC 27606-5226; Comments? Questions?, title: National Weather Service, link: https://forecast.weather.gov/zipcity.php?inputstring=Raleigh,NC&Go2=Go], [snippet: R

RatelimitException: https://duckduckgo.com/ 202 Ratelimit